In [ ]:
import sys
path = r'C:\Users\infin\OneDrive\Documents\GitHub\aimodels\projects\tooling'
sys.path.insert(0, path)

In [ ]:

repo = "rahular/simple-wikipedia"
simple = datasets.load_dataset(repo)
simple['train'][668959]['text']

In [ ]:
tokenizer.decode(342)

In [ ]:
tokenizer(simple['train'][668959]['text']).input_ids

In [ ]:
print(tokenizer.decode(28705))
print(tokenizer.decode(28750))
print(tokenizer.decode(28725))
print(tokenizer.decode([28725, 28705]))
print(tokenizer(","))
print(tokenizer(", "))
print(tokenizer("2 ,"))


In [ ]:
import datasets
import pandas as pd

from transformers import AutoTokenizer
mistral = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(mistral, padding_side = "right")
tokenizer.pad_token = tokenizer.eos_token

repo = "rahular/simple-wikipedia"
simple = datasets.load_dataset(repo)
rows = [{'tokens': [tokenizer.bos_token_id]}]
max_tokens = 512
#line_break_token = tokenizer("\n").input_ids[-1]
period_token = tokenizer(".").input_ids[-1]
comma_token = tokenizer(",").input_ids[-1]
for i, row in enumerate(simple['train']):
    #print(f"Processing row {i}.")
    last_row = rows[-1]
    if len(last_row) > max_tokens:
        print("that wasn't supposed to happen")
        break
        #pass

    tokens = tokenizer(row['text']).input_ids[1:]
    if len(last_row['tokens']) + len(tokens) > max_tokens:
        #print(f"Closing out row with {len(last_row['tokens'])} tokens.")
        rows.append({'tokens': [tokenizer.bos_token_id]})
        last_row = rows[-1]

    splitting_tokens = [period_token, comma_token, 28723, 28725, 342] # seems to be some kind of numeric comma?
    splitting_token = splitting_tokens.pop(0)

    loops = 0
    while len(tokens) > max_tokens: # look for ways to split the row
        loops += 1
        if loops > 100:
            raise ValueError("Suspected infinite loop for row " + str(i))
            break

        best_split = 0
        for j, token in enumerate(tokens): # look for the splitting token
            if token == splitting_token:
                tentative_length = len(last_row['tokens']) + len(tokens[:j+1])
                if tentative_length > max_tokens:
                    if best_split == 0:
                        rows.append({'tokens': [tokenizer.bos_token_id]}
                        break # close out the row and restart the loop
                    rows[-1]['tokens'].append(tokens[:best_split])
                    tokens = tokens[best_split:]
                    #print(f"Closing out row with {len(last_row['tokens'])} tokens.")
                    rows.append({'tokens': [tokenizer.bos_token_id]})
                    break # this will break out of the enumerate loop 
                else:
                    best_split = j+1

        if best_split > 0:
            rows[-1]['tokens'] += tokens[:best_split]
            tokens = tokens[best_split:]
        else:
            if len(splitting_tokens)>0:
                splitting_token = splitting_tokens.pop(0)
            else:
                raise ValueError("Unable to find splitting token for row " + str(i))
                break

    last_row['tokens'] += tokens

                
    # that works until we hit a very long row, then it breaks


df = pd.DataFrame(rows)
ds = datasets.Dataset.from_pandas(df)
ds # that takes less than three minuts to run and I'm guessing it's >90% accurate with the document breaks.

In [ ]:
print(tokenizer("2."))
print(tokenizer("test."))


In [ ]:
import datasets
import pandas as pd

from transformers import AutoTokenizer
mistral = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(mistral, padding_side = "right")
tokenizer.pad_token = tokenizer.eos_token

repo = "rahular/simple-wikipedia"
simple = datasets.load_dataset(repo)
rows = [{'tokens': [tokenizer.bos_token_id]}]
max_tokens = 512
split_tokens_ref = [842, 28723, 1200, 28725, 342] # both variants of period, both variants of comma, and pipe, so far
for i, row in enumerate(simple['train']):
    #print(f"Processing row {i}.")
    last_row = rows[-1]
    if len(last_row) > max_tokens: # if the current last row is too long, 
        raise ValueError("0: current row is too long at input row " + str(i))

    tokens = tokenizer(row['text']).input_ids[1:]
    if len(last_row['tokens']) + len(tokens) > max_tokens:
        #print(f"Closing out row with {len(last_row['tokens'])} tokens.")
        rows.append({'tokens': [tokenizer.bos_token_id]})
        last_row = rows[-1]

    splitting_tokens = split_tokens_ref.copy()
    splitting_token = splitting_tokens.pop(0)
    loops = 0
    while len(last_row) + len(tokens) > max_tokens: # look for ways to split the row
        loops += 1
        if loops > 100:
            raise ValueError("Suspected infinite loop for row " + str(i))
            break

        best_split = 0
        for j, token in enumerate(tokens): # look for the splitting token
            if token == splitting_token:
                tentative_length = len(last_row['tokens']) + len(tokens[:j+1])
                if tentative_length > max_tokens:
                    if best_split == 0:
                        if len(last_row['tokens']) > 512:
                            raise ValueError("1: current row is too long at input row " + str(i))
                        rows.append({'tokens': [tokenizer.bos_token_id]})
                        break # close out the row and restart the loop
                    rows[-1]['tokens'].append(tokens[:best_split])
                    tokens = tokens[best_split:]
                    #print(f"Closing out row with {len(last_row['tokens'])} tokens.")
                    rows.append({'tokens': [tokenizer.bos_token_id]})
                    last_row = rows[-1]
                    break # this will break out of the enumerate loop 
                else:
                    best_split = j+1

        if best_split > 0:
            rows[-1]['tokens'] += tokens[:best_split]
            if len(last_row['tokens']) > 512:
                raise ValueError("2: current row is too long at input row " + str(i))
            tokens = tokens[best_split:]
        else:
            if len(splitting_tokens)>0:
                splitting_token = splitting_tokens.pop(0)
            else:
                raise ValueError("Unable to find splitting token for row " + str(i))

    last_row['tokens'] += tokens
    if len(last_row['tokens']) > 512:
        raise ValueError("3: current row is too long at input row " + str(i))

                
    # that works until we hit a very long row, then it breaks


df = pd.DataFrame(rows)
ds = datasets.Dataset.from_pandas(df)

In [1]:
import datasets
import pandas as pd

from transformers import AutoTokenizer
mistral = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(mistral, padding_side = "right")
tokenizer.pad_token = tokenizer.eos_token

repo = "rahular/simple-wikipedia"
simple = datasets.load_dataset(repo)

def create_new_row():
    return {'tokens': [tokenizer.bos_token_id]}



Found cached dataset parquet (C:/Users/infin/.cache/huggingface/datasets/rahular___parquet/rahular--simple-wikipedia-e8322ace21be7483/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Processing input row 0


In [29]:
r = 668971
print(simple["train"][r]['text'])
print(tokenizer(simple["train"][r]['text']).input_ids)
print(len(tokenizer(simple["train"][r]['text']).input_ids))

Macaroni Factory | Macaroni Factory | Madan-e Surameh | Madar Dokhtar | Madar Soleyman | Madavan | Madeh Banan | Madevan | Madkhun | Mah Farrokhan | Mah Kord | Mah Salari | Mahal Ahdas-e Sad Rudbal | Mahall ol Din | Mahalleh-ye Akbari | Maharak | Maharlu Kohneh | Maharlu Now | Maharzir | Mahdiyeh | Mah-e Firuzan | Mahjan | Mahjanabad | Mahlacheh | Mahmansaray Shomareh-ye Do | Mahmeleh | Mahmudabad | Mahmudabad | Mahmudabad | Mahmudabad | Mahmudabad | Mahmudabad | Mahmudabad | Mahmudabad | Mahmudabad | Mahmudabad | Mahmudabad | Mahmudabad-e Do Dang | Mahmudabad-e Olya | Mahmudabad-e Olya | Mahmudabad-e Seh Dang | Mahmudabad-e Sofla | Mahmudabad-e Yek Dang | Main | Majdabad | Majdabad | Makuyeh | Mal Hajji | Malavan | Malay-e Anbar | Mal-e Ahmadi | Mal-e Mahmud | Malekabad | Malekabad | Malekabad | Malekabad | Malekabad | Malekabad | Malekabad | Malicheh Sheykh | Malicheh | Malyan | Mambalu | Mamu | Mangun | Manian | Mansurabad | Mansurabad | Mansurabad | Mansurabad | Mansurabad | Mansur

In [40]:
print(simple["train"][r-14]['text']) # I'm tempted to say anything with ten or more |s in it should be thrown out.

A list of cities, towns and villages in Fars Province of southern Iran:


In [27]:
rows = [create_new_row()]

max_tokens = 512
split_tokens_ref = (842, 28723, 1200, 28725, 342)

def try_add_tokens(rows, next_tokens, max_tokens = 512, splitting_tokens = (842, 28723, 1200, 28725, 342)):
    current_tokens = rows[-1]['tokens']
    if len(current_tokens) + len(next_tokens) <= max_tokens:
        rows[-1]['tokens'] += next_tokens
        return []
    else:
        for token in splitting_tokens: # tokens in order of preference
            if token in next_tokens:
                idx = next_tokens.index(token)
                if len(current_tokens) + idx <= max_tokens:
                    rows[-1]['tokens'] += next_tokens[:idx+1]
                    return next_tokens[idx+1:]
    # if we get here, we need to close out the row and start a new one
    rows.append(create_new_row())
    return next_tokens

for i, row in enumerate(simple['train']):
    next_tokens = tokenizer(row['text']).input_ids[1:]
    loops = 0
    while len(next_tokens) > 0:
        loops += 1
        if loops > 5000:
            raise ValueError("Suspected infinite loop for row " + str(i))

        #next_tokens = try_add_tokens(rows, next_tokens, debug = i==13814)
        next_tokens = try_add_tokens(rows, next_tokens)

ValueError: Suspected infinite loop for row 668971

In [2]:




for i, row in enumerate(simple['train']):
    print("Processing input row " + str(i))
    next_tokens = tokenizer(row['text']).input_ids[1:]
    loops = 0
    while len(next_tokens) > 0:
        loops += 1
        if loops > 100:
            raise ValueError("Suspected infinite loop for row " + str(i))
        next_tokens = try_add_tokens(rows, next_tokens)

df = pd.DataFrame(rows)
ds = datasets.Dataset.from_pandas(df)

In [ ]:
rows = [create_new_row()]
rows[-1]["tokens"]

In [ ]:
[1, 2, 3].find(4)

In [ ]:
for i in range(len(df)):
    row = df.iloc[i]
    if type(row['tokens']) != list:
        print(f"Row {i} is not a list.")
    if len(row['tokens']) > max_tokens:
        print(f"Row {i} is too long at {len(row['tokens'])} tokens.")

In [ ]:
import torch
import random
from collections import defaultdict

class BucketSampler(torch.utils.data.Sampler):
    
    def __init__(self, lengths, buckets=(50,500,50), shuffle=True, batch_size=32, drop_last=False):
        
        super().__init__(lengths)
        
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.drop_last = drop_last
        
        assert isinstance(buckets, tuple)
        bmin, bmax, bstep = buckets
        assert (bmax - bmin) % bstep == 0
        
        buckets = defaultdict(list)
        for i, length in enumerate(lengths):
            if length > bmin:
                bucket_size = min((length // bstep) * bstep, bmax)
                buckets[bucket_size].append(i)
                
        self.buckets = dict()
        for bucket_size, bucket in buckets.items():
            if len(bucket) > 0:
                self.buckets[bucket_size] = torch.tensor(bucket, dtype=torch.int, device='cpu')
        
        # call __iter__() to store self.length
        self.__iter__()
            
    def __iter__(self):
        print("__iter__ got called")
        if self.shuffle == True:
            for bucket_size in self.buckets.keys():
                # shuffle the indices in each bucket
                self.buckets[bucket_size] = self.buckets[bucket_size][torch.randperm(self.buckets[bucket_size].nelement())]
                
        batches = []
        for bucket in self.buckets.values(): # iterate over the buckets
            curr_bucket = torch.split(bucket, self.batch_size) # split the bucket into batches
            if len(curr_bucket) > 1 and self.drop_last == True: # handle drop_last
                if len(curr_bucket[-1]) < len(curr_bucket[-2]):
                    curr_bucket = curr_bucket[:-1]
            batches += curr_bucket # add the batches to the list of all batches
            
        self.length = len(batches) # the length of a Sampler might be used to let the DataLoader know how many batches it should expect
        
        if self.shuffle == True:
            random.shuffle(batches) # shuffle the batches
            
        return iter(batches) # return an iterator over the batches
    
    def __len__(self):
        return self.length

# add a length column to the dataset
simple['train'] = simple['train'].map(lambda x: {'lengths': len(x['text'].split())})
simple['train']
bucket_dataloader = torch.utils.data.DataLoader(simple['train'], batch_sampler=BucketSampler(simple['train']['lengths'], batch_size=32, shuffle=True, drop_last=False))


In [ ]:
buckets = BucketSampler(simple['train']['lengths']).buckets
for k, v in buckets.items():
    print(k, v.shape)

# So I'm actually a bit confused.  Oh I see, the code Copilot wrote for me splits on spaces.  So this is kind of an approximate tokenization.
# So the way this works is that the sampler (1) receives the entire datas set and (2) I guess handles all the shuffling and batching itself?

# Okay, so how did that rabbit hole all come up?  It was because I was trying to understand exactly where the tokenization can fit into the process, and how we trade off efficiency and flexibility.
# Oh...what happens if we just wrap the Hugging Face dataset?  Okay it spits out a dictionary.

# So what's next?  I think you essentially always want to using a torch-tokenized dataset as the input for the model, because we have demonstrated that even converting lists of ints to tensors takes non-negligible time.
# In addition, you can't do any bucketing or anything like that if you don't have the tokenized dataset.
# Now there's a downside, which is that in order to tokenize the dataset with torch, you pretty much have to pad it.

# Anyways, regardless of that we have some preprocessing to do.  The simple way is to just bucket everything and pretend document breaks don't matter.



In [ ]:
from transformers import AutoTokenizer
gpt2t = AutoTokenizer.from_pretrained('gpt2')
mistralt = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tallies = {
    'gpt2_tokens': 0,
    'mistral_tokens': 0,
    'chars': 0
}
for row in simple['train']:
    text = row['text']
    tallies['gpt2_tokens'] += len(gpt2t.tokenize(text))
    tallies['mistral_tokens'] += len(mistralt.tokenize(text))
    tallies['chars'] += len(text)

print(tallies) # it's actually around 2.7!

In [ ]:
37.5/14/1


In [ ]:
from torch.utils.data import DataLoader
hf_loader = DataLoader(simple['train'], batch_size=32, shuffle=True, drop_last=False)

for batch in hf_loader:
    print(batch)
    break

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def dynamic_length_collate_fn(lst):
    text = [x["text"] for x in lst]
    max_len = max(len(x) for x in lst)
    return tokenizer(
        text,
        padding="longest",
        return_tensors="pt",
    ).input_ids

from torch.utils.data import DataLoader
dataloader = DataLoader(simple["train"], batch_size = 32, collate_fn = dynamic_length_collate_fn)
for i, batch in enumerate(dataloader):
    print(batch.shape)
    if i > 10:
        break

In [ ]:
# Okay, so apparently there is a default "RandomSampler"
# Note: RandomSampler(train_ds,replacement=True).  You pass both the Dataset itself and a sampler that points to the Dataset to the loader.
class weightedsampler(Sampler):
    
    def __init__(self,dataset):
        
        self.indices = list(range(len(dataset)))
        self.num_samples = len(dataset)
        self.label_to_count = dict(Counter(dataset.bins))
        weights = [1/self.label_to_count[i] for i in dataset.bins]
        
        self.weights = torch.tensor(weights,dtype=torch.double)
        
    def __iter__(self):
        count = 0
        index = [self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True)]
        while count < self.num_samples:
            yield index[count]
            count += 1
    
    def __len__(self):
        return self.num_samples
        

# So it looks like when the sampler is initialized on the Dataset, it analyzes the Dataset, inspects a property called dataset.bins (which I think is custom?) in order to save a tensor of weights.
# the actual weighting logic is used from torch.multinomial which seems to be built for this sort of thing.  Then it defines __iter__ (with yield) and __len__ which I assume are needed for it to be a Sampler.
# Okay, and it sounds like Samplers are used only with map-style datasets.
# Alright, if you set both batch_size and sampler to None, then the collate_fn sees an individual data sample, which doesn't sound useful.
# Okay it sounds like maybe I should always set pin_memory to True?


In [ ]:
#https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler

In [ ]:
def quick_estimate_tokens(ds, field="text", already_tokenized = False, chars_per_token=4):
    tally = 0
    max_len = 0
    lengths = {}
    for row in ds:
        l = len(row[field])
        tally += l
        lengths[l] = lengths.get(l, 0) + 1
        if l > max_len:
            max_len = l

    print(f'{int(tally):_}')
    print(f'Max length: {max_len}, estimated tokens: {int(max_len / chars_per_token):_}')
    lengths = list(lengths.items())
    lengths.sort(reverse=True)
    return int(tally), lengths

total, length = quick_estimate_tokens(simple['train'])


In [ ]:
import pandas as pd
df = pd.DataFrame(length, columns=['Length', 'Count'])
#histogram
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
def map_tokenize_as_list(x):
    return tokenizer(x['text'])



list_tokenized = simple['train'].map(map_tokenize_as_list, batched=True, remove_columns=['text']) # That takes 30 seconds to tokenize



In [ ]:
list_tokenized

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
def map_tokenize_as_tensors(x):
    return tokenizer(x['text'], return_tensors="pt", padding=True, truncation=True, max_length=512)

tensor_tokenized = simple['train'].map(map_tokenize_as_tensors, batched=True, remove_columns=['text']) # That takes 2 whole minues to tokenize

In [ ]:
import time
start = time.time()
for batch in tokenizing_dataloader:
    pass
print(time.time() - start)

In [ ]:
quick_estimate_tokens(tokenized['train'], field="input_ids")

In [ ]:
def token_profile(ds, field, print_every = None)
    """ This should operate on an already-tokenized dataset. """
    freqs = {}
    # can we do this with datasets.map?
    n = len(ds)
    print(f"Processing {n} examples")
    for idx, ex in enumerate(ds):
        if print_every is not None and idx % print_every == 0:
            print(f"Processed {idx} examples")

        if 
        for t in tokens:
            print(t)
            if t in freqs:
                freqs[t] += 1
            else:
                freqs[t] = 1

        break

    total = sum(freqs.values())
    desc = [(tknzr.decode(k), v/total) for k, v in freqs.items()]
    return total, desc, freqs


import datasets
simple = "rahular/simple-wikipedia"
simple = datasets.load_dataset(simple)
simple

from data import WrappedHuggingFaceTokenizer
tokenizer = WrappedHuggingFaceTokenizer(WrappedHuggingFaceTokenizer.MISTRAL)

total, ordered, freqs = token_profile(simple['train'], 'text', tokenizer, print_every = 100_000)

In [ ]:
freqs

In [ ]:
# Okay.  So, question.  Attention mask.  Ah okay, so Hugging Face expects the data collator to take care of the masking.
# And I guess the reason so many tokenizers don't have default pad_tokens is that they were trained without them and Hugging Face wants to serve up exactly what was used in training.
# So now I understand "why the fence is there"; now how do I change it?
# So there's a bit of a dilemma here because I want to be able to use Hugging Face's tokenizers, but I don't want to depend on their overall data collation strategy.
# I could wrap their tokenizer in my own class that changes eos_token to -100.  Ummm...will the embedding choke on that?  I believe it will.  Well, a pretrained embedding will, anyway.
# There's going to be a potentially serious issue here where comparisons to pretrained models are really hard.  But maybe I can set up a wrapper for that.
# So, question about padding strategies.  I'm currently using the nn.Transformers method that creates causal attention masks.
# One thing I could just do is assume -100 is always the padding token.  If I do that, then I also have to...well, hold on.  If we left-pad, then things get weird and I have to use an attention mask.
# But I think if I right-pad, everything is fine?  You'll waste a bit of time calculating attention for the padded tokens in batches where not eveyrthing is the same length, but I think you have to do that anyway.
# I think left-padding is a strategy for .generate()?  I think that's it.
